# <h1 align=center>**`Data Engineering - Movies Score - Proyecto Individual`**</h1>

¡Bienvenidos al primer proyecto individual de la etapa de labs! En esta ocasión, deberán hacer un trabajo situándose en el rol de un ***Data Engineer***.  

<hr>  

## **Descripción del problema (Contexto y rol a desarrollar)**

## Contexto

`Application Programming Interface`  es una interfaz que permite que dos aplicaciones se comuniquen entre sí, independientemente de la infraestructura subyacente. Son herramientas muy versátiles que permiten por ejemplo, crear pipelines facilitando mover y brindar acceso simple a los datos que se quieran disponibilizar a través de los diferentes endpoints, o puntos de salida de la API.

Hoy en día contamos con **FastAPI**, un web framework moderno y de alto rendimiento para construir APIs con Python.
<p align=center>
<img src = 'https://i.ibb.co/9t3dD7D/blog-zenvia-imagens-3.png' height=250><p>

## Rol a desarrollar

Como parte del equipo de data de una empresa, el área de análisis de datos le solicita al área de Data Engineering (usted) ciertos requerimientos para el óptimo desarrollo de sus actividades. Usted deberá elaborar las *transformaciones* requeridas y disponibilizar los datos mediante la *elaboración y ejecución de una API*.



## **Propuesta de trabajo (requerimientos de aprobación)**

**`Transformaciones`**:  El analista de datos requiere estas, ***y solo estas***, transformaciones para sus datos:


+ Generar campo **`id`**: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = **`as123`**)

+ Los valores nulos del campo rating deberán reemplazarse por el string “**`G`**” (corresponde al maturity rating: “general for all audiences”

+ De haber fechas, deberán tener el formato **`AAAA-mm-dd`**

+ Los campos de texto deberán estar en **minúsculas**, sin excepciones

+ El campo ***duration*** debe convertirse en dos campos: **`duration_int`** y **`duration_type`**. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

<br/>

**`Desarrollo API`**:  Para disponibilizar los datos la empresa usa el framework ***FastAPI***. El analista de datos requiere consultar:

+ Cantidad de veces que aparece una keyword en el título de peliculas/series, por plataforma

+ Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año

+ La segunda película con mayor score para una plataforma determinada, según el orden alfabético de los títulos.

+ Película que más duró según año, plataforma y tipo de duración

+ Cantidad de series y películas por rating

<br/>


**`Deployment`**: La empresa suele usar [Deta](https://www.deta.sh/?ref=fastapi) (no necesita dockerizacion) para realizar el deploy de sus aplicaciones. Sin embargo, también puede usar [Railway](https://railway.app/) y [Render](https://render.com/docs/free#free-web-services) (necesitan dockerizacion).
<br/>

<br/>

**`Video`**: El Tech Lead que le delegó esta tarea quiere darle un feedback sobre el trabajo realizado. Para esto, le pide que sintetice en un video de ***5 minutos*** su trabajo resaltando cómo ayuda el mismo a los analistas de datos.

<sub> **Spoiler: Para lograr esto DEBE mostrarle al TL las consultas requeridas en funcionamiento desde la API**. <sub/>

<br/>

## **Criterios de evaluación**

**`Código`**: Prolijidad de código, uso de clases y/o funciones, en caso de ser necesario, código comentado. 

**`Repositorio`**: Nombres de archivo adecuados, uso de carpetas para ordenar los archivos, README.md presentando el proyecto y el trabajo realizado

**`Cumplimiento`** de los requerimientos de aprobación indicados en el apartado `Propuesta de trabajo`

NOTA: Recuerde entregar el link de acceso al video. Puede alojarse en YouTube, Drive o cualquier plataforma de almacenamiento. **Verificar que sea de acceso público**.

<br/>

## **Fuente de datos**

+ Podrán encontrar los archivos con datos en la carpeta Datasets, en este mismo repositorio.<sup>*</sup>
<br/>

## **Material de apoyo**

Imagen Docker con Uvicorn/Guinicorn para aplicaciones web de alta performance:

+ https://hub.docker.com/r/tiangolo/uvicorn-gunicorn-fastapi/ 

+ https://github.com/tiangolo/uvicorn-gunicorn-fastapi-docker

FAST API Documentation:

+ https://fastapi.tiangolo.com/tutorial/

"Prolijidad" del codigo:

+ https://pandas.pydata.org/docs/development/contributing_docstring.html

<br/>

## **Deadlines importantes**

+ Apertura de formularios de entrega de proyectos: **Miercoles 18, 15:00hs gmt -3**

+ Cierre de formularios de entrega de proyectos: **Viernes 20, 12:00hs gmt-3**
  
+ Demo por parte del estudiante: **Viernes 20, 16:00hs gmt-3** 

(Se escogera entre l@s estudiantes aquel que represente de **forma global** todos los criterios de evaluacion esperados, para que sirva de inspiracion a sus compañer@s)

## `Disclaimer`
De parte del equipo de Henry se aclara y remarca que el fin de los proyectos propuestos es exclusivamente pedagógico, con el objetivo de realizar simular un entorno laboral, en el cual se trabajan diversas temáticas ajustadas a la realidad. No reflejan necesariamente la filosofía y valores de la organización. Además, Henry no alienta ni tampoco recomienda a los alumnos y/o cualquier persona leyendo los repositorios (y entregas de proyectos) que tomen acciones con base a los datos que pudieran o no haber recabado. Toda la información expuesta y resultados obtenidos en los proyectos nunca deben ser tomados en cuenta para la toma real de decisiones (especialmente en la temática de finanzas, salud, política, etc.).

## **Desarrollo de las transformaciones**

### ``Importación de librerías``

In [1]:
# Importamos las librerías pandas y numpy para realizar las transformaciones
import pandas as pd
import numpy as np
# Mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## ``Carga de datasets``

In [20]:
amazon = pd.read_csv('./Datasets/amazon_prime_titles-score.csv')
disney = pd.read_csv('./Datasets/disney_plus_titles-score.csv')
hulu = pd.read_csv('./Datasets/hulu_titles-score (2).csv')
netflix = pd.read_csv('./Datasets/netflix_titles-score.csv')

Antes de realizar las transformaciones consideraremos un diccionario cuyas llaves serán los nombre de los datasets y sus respectivos valores son la importación de los archivos csv correspondiente, tal como sigue:

In [21]:
data_dictionary = {
    'amazon': amazon,
    'disney': disney,
    'hulu': hulu,
    'netflix': netflix
}

Nota: La metodología va ha consistir en aplicar las transformaciones a todos los datasets y en el orden que nos proponen. Una vez aplicada dichas transformaciones concatenaremos los dataframes para exportarlo en un archivo JSON con el nombre ``movies_scores.json``. Finalemente, desarrollaremos una API usando FastAPI y realizaremos las consultas basadas en los datos transformados ``movies_scores.json``.

### **``Transformación 1:``**
Generar campo **`id`**: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = **`as123`**)

In [22]:
# Funcion para agregar/concatenar la letra (string) inicial del nombre de la plataforma en show_id
def generate_id_show(dict_data):
    for df_name , df in zip(dict_data.keys(), dict_data.values()):
        df['show_id'] = df_name[0] + df['show_id']
        dict_data[df_name] = df    
    return dict_data

In [23]:
# Aplicación de la transformación 1
data_dictionary = generate_id_show(data_dictionary)

In [27]:
data_dictionary['amazon'].head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,as1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,99


### **``Transformación 2:``**
Los valores nulos del campo rating deberán reemplazarse por el string “**`G`**” (corresponde al maturity rating: “general for all audiences”

In [28]:
# Veamos cuantos valores nulos serán afectados por esta transformación
amazon['rating'].isnull().sum() # 337 valores

337

In [29]:
# Funcion para reemplazar los valores nulos de la columna rating por 'g' en los datasets
def fillna_initial_df_name(dict_data):
    for df_name,df in  zip(dict_data.keys(),dict_data.values()):
        df['rating'] = df['rating'].fillna('g')
        dict_data[df_name] = df
    return dict_data

In [30]:
# Aplicación de la transformación 2
data_dictionary = fillna_initial_df_name(data_dictionary)

In [32]:
# Veamos como queda despues de aplicar la funcion de transformación
print('Datos nulos despues de la transformaación:',data_dictionary['amazon']['rating'].isnull().sum()) # 0 datos nulos
print('Datos únicos despues de la transformaación:',data_dictionary['amazon']['rating'].unique())

Datos nulos despues de la transformaación: 0
Datos únicos despues de la transformaación: ['g' '13+' 'ALL' '18+' 'R' 'TV-Y' 'TV-Y7' 'NR' '16+' 'TV-PG' '7+' 'TV-14'
 'TV-NR' 'TV-G' 'PG-13' 'TV-MA' 'G' 'PG' 'NC-17' 'UNRATED' '16' 'AGES_16_'
 'AGES_18_' 'ALL_AGES' 'NOT_RATE']


### **``Transformación 3:``**
De haber fechas, deberán tener el formato **`AAAA-mm-dd`**

In [33]:
def change_date_added_datetime(dict_data):
    for df_name, df in zip(dict_data.keys(), dict_data.values()):
        df['date_added'] = pd.to_datetime(df['date_added'].str.strip(),  format='%B %d, %Y').dt.strftime('%Y-%m-%d')
        dict_data[df_name] = df
    return dict_data

In [34]:
# Aplicación de la transformación 3
data_dictionary = change_date_added_datetime(data_dictionary)

In [37]:
# Veamos los cinco primeros valores de la columna date_added con el formato "AAAA-mm-dd"
data_dictionary['amazon']['date_added'].head(5)

0    2021-03-30
1    2021-03-30
2    2021-03-30
3    2021-03-30
4    2021-03-30
Name: date_added, dtype: object

### **``Transformación 4:``**
Los campos de texto deberán estar en **minúsculas**, sin excepciones

In [38]:
# Funcion para convertir a minúscula todas las columna de tipo Objeto y/o con valores strings
def convert_to_lower(dict_data):
    for df_name, df in zip(dict_data.keys(), dict_data.values()):
        object_columns_list= df.dtypes[df.dtypes == 'object'].index.to_list()
        for col in object_columns_list:
            df[col] = df[col].str.lower()
        dict_data[df_name] = df
    return dict_data

In [39]:
# Aplicacion de la transformación 4
data_dictionary = convert_to_lower(data_dictionary)

In [40]:
# Veamos como quedaron los datos string en minúscula
data_dictionary['amazon'].head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,99
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,37
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,20


### **``Transformación 5:``**
El campo ***duration*** debe convertirse en dos campos: **`duration_int`** y **`duration_type`**. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [41]:
# Verifiquemos datos atípicos que quizá no puedan convertir al tipo entero
disney['duration'].unique()
hulu['duration'].unique()
netflix['duration'].unique()
amazon['duration'].unique() # Todos se pueden convertir a tipo entero (int), no hay caracteres alfabéticos u otros que no sean numéricos.

array(['113 min', '110 min', '74 min', '69 min', '45 min', '52 min',
       '98 min', '131 min', '87 min', '92 min', '88 min', '93 min',
       '94 min', '46 min', '96 min', '1 season', '104 min', '62 min',
       '50 min', '3 seasons', '2 seasons', '86 min', '36 min', '37 min',
       '103 min', '9 min', '18 min', '14 min', '20 min', '19 min',
       '22 min', '60 min', '6 min', '54 min', '5 min', '84 min',
       '126 min', '125 min', '109 min', '89 min', '85 min', '56 min',
       '40 min', '111 min', '33 min', '34 min', '95 min', '99 min',
       '78 min', '4 seasons', '77 min', '55 min', '53 min', '115 min',
       '58 min', '49 min', '135 min', '91 min', '64 min', '59 min',
       '48 min', '122 min', '90 min', '102 min', '65 min', '114 min',
       '136 min', '70 min', '138 min', '100 min', '480 min', '4 min',
       '30 min', '152 min', '68 min', '57 min', '7 seasons', '31 min',
       '151 min', '149 min', '9 seasons', '141 min', '121 min', '79 min',
       '140 min', '51 min'

**``Observación:``** Los datos no parecen estar normalizados podemos notar que existen valores de la estrutura ``XXX season`` y ``XXX seasons`` lo que deberían implicar la misma unidad de medida de temporada de las series. Por esta razón aplicaremos una transformación adicional de reemplazo de los valores del tipo ``XXX seasons`` por ``XXX season`` para que quede normalizado en la columna ``duration_type``

In [43]:
# Función para convertir 
def normalize_duration(dict_data):
    for df_name, df in zip(dict_data.keys(), dict_data.values()):
        # Separación por ' ' y expansión en columnas de los valores separados creando las columnas duration_int y duration_type
        df[['duration_int', 'duration_type']] = df['duration'].str.split(' ', expand=True)
        # Reeemplazo de los valores 'seasons' por 'season'
        df['duration_type'] = df['duration_type'].str.replace('seasons', 'season')
        # Converción al tipo entero de los datos en la columna duration_int
        df['duration_int'] = pd.to_numeric(df['duration_int'], downcast='integer', errors='coerce')
        # df['duration_int'] = df['duration_int'].astype(int)
        dict_data[df_name] = df
    return dict_data

In [44]:
# Aplicación de la transformación 5
data_dictionary = normalize_duration(data_dictionary)

In [46]:
# Veamos cómo quedó despues de la transformación
data_dictionary['amazon'].head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,99,113,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,37,110,min


### **``Concatenación de dataframes``**

In [47]:
# Función para concatenar los dataframes transformados
def concat_dataframes(dict_data):
    movies = pd.concat([dict_data['amazon'], dict_data['disney'], dict_data['hulu'], dict_data['netflix']],axis=0)
    return movies

In [49]:
# Aplicación de la concatenación quedando como dataframe concatenado movies_scores_df
movies_scores_df = concat_dataframes(data_dictionary)

In [50]:
# Función para reordenar las columnas del dataframe
def reorder_cols(df):
    reordered_cols = ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added','release_year', 'rating', 'duration', 'duration_int', 'duration_type', 'listed_in', 'description','score']
    df = df[reordered_cols]
    return df

In [51]:
# Aplicación del reordenamiento
movies_scores_df = reorder_cols(movies_scores_df)

In [52]:
# Veamos la información general del dataframe final
movies_scores_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        22998 non-null  object 
 1   type           22998 non-null  object 
 2   title          22998 non-null  object 
 3   director       14739 non-null  object 
 4   cast           17677 non-null  object 
 5   country        11499 non-null  object 
 6   date_added     13444 non-null  object 
 7   release_year   22998 non-null  int64  
 8   rating         22998 non-null  object 
 9   duration       22516 non-null  object 
 10  duration_int   22516 non-null  float64
 11  duration_type  22516 non-null  object 
 12  listed_in      22998 non-null  object 
 13  description    22994 non-null  object 
 14  score          22998 non-null  int64  
dtypes: float64(1), int64(2), object(12)
memory usage: 2.8+ MB


### **``Exportación de la data transformada``**

In [55]:
def save_as_json(df):
    # path_file = Path('app/src/db/movies_scores.json')
    # Exportar a JSON: convierte el DataFrame a JSON con orientación 'records' (lista de diccionarios)
    df_json = df.to_json(orient='records')
    # Guarda el JSON en un archivo .json
    with open('../app/db/movies_scores.json','w') as file:
        file.write(df_json)


In [56]:
# Ejecutando la funcion para guardar el archivo "movies_scores.json"
save_as_json(movies_scores_df)

<h3 align='center'><b>Gracias por leer</b></h3>
<!-- <span align='center'><b>Roy Quillca Pacco</b> <button src='https://google.com'>royquillca</button></span> -->